In [1]:
import pandas as pd
import sqlite3

In [13]:
#Conecto a la base de datos
connection = sqlite3.connect("./sql-murder-mystery.db")

In [15]:
#Obtengo el cursor para hacer las queries
game_cursor = connection.cursor()

In [62]:
#Creo la funcion para leer datos y pasar a un dataframe de pandas

def sql_query(query):
    game_cursor.execute(query)
    ans = game_cursor.fetchall()
    names = [description[0] for description in game_cursor.description]
    #Modifico la funcion para poder ver las columnas enteras 
    for row in ans:
        print([str(col) for col in row])  # Convierte todas las columnas a string para evitar truncamientos
    
    return pd.DataFrame(ans, columns=names)


In [53]:
#query para ver los nombres de las tablas
query = ''' 
SELECT name 
FROM sqlite_master
where type = 'table'
'''
sql_query(query)


['crime_scene_report']
['drivers_license']
['person']
['facebook_event_checkin']
['interview']
['get_fit_now_member']
['get_fit_now_check_in']
['income']
['solution']


,name
0,crime_scene_report
1,drivers_license
2,person
3,facebook_event_checkin
4,interview
5,get_fit_now_member
6,get_fit_now_check_in
7,income
8,solution


In [25]:
#Estructura de 'crime_scene_report' table
query = '''
SELECT * 
FROM crime_scene_report;
'''
sql_query(query)

,date,type,description,city
0,20180115,robbery,A Man Dressed as Spider-Man Is on a Robbery Spree,NYC
1,20180115,murder,Life? Dont talk to me about life.,Albany
2,20180115,murder,"Mama, I killed a man, put a gun against his he...",Reno
3,20180215,murder,REDACTED REDACTED REDACTED,SQL City
4,20180215,murder,Someone killed the guard! He took an arrow to ...,SQL City
...,...,...,...,...
1223,20180430,bribery,\n,Garden Grove
1224,20180430,fraud,‘Why not?’ said the March Hare.\n,Houma
1225,20180430,assault,\n,Fontana
1226,20180501,assault,be NO mistake about it: it was neither more no...,Trenton


In [36]:
# Actualizo la columna 'date' al formato europeo
game_cursor.execute("""
    UPDATE crime_scene_report
    SET date = SUBSTR(date, 7, 2) || '-' || SUBSTR(date, 5, 2) || '-' || SUBSTR(date, 1, 4)
""")

## Pistas
Sabemos que el crimen fue un asesinato ➡️ Filtrar por type: murder  
Ocurrio el 15 de enero de 2018 ➡️ Filtrar date   
Sucedio en SQL City ➡️ Filtrar en City: SQL City  

In [55]:
#Filtro por type: murder, date: 15-01-2018, city =SQL City
query='''
SELECT *
FROM crime_scene_report
WHERE type = "murder"
	AND date = "15-01-2018"
	AND city = "SQL City";
'''
sql_query(query)

['15-01-2018', 'murder', 'Security footage shows that there were 2 witnesses. The first witness lives at the last house on "Northwestern Dr". The second witness, named Annabel, lives somewhere on "Franklin Ave".', 'SQL City']


,date,type,description,city
0,15-01-2018,murder,Security footage shows that there were 2 witne...,SQL City


Luego tenemos que ver quienes son los testigos involucrados: dice que el primer testigo vivio por ultima vez en "Northwestern Dr", y el segundo en "Franklin Ave".

In [45]:
query='''
SELECT *
FROM person;
'''
sql_query(query)

,id,name,license_id,address_number,address_street_name,ssn
0,10000,Christoper Peteuil,993845,624,Bankhall Ave,747714076
1,10007,Kourtney Calderwood,861794,2791,Gustavus Blvd,477972044
2,10010,Muoi Cary,385336,741,Northwestern Dr,828638512
3,10016,Era Moselle,431897,1987,Wood Glade St,614621061
4,10025,Trena Hornby,550890,276,Daws Hill Way,223877684
...,...,...,...,...,...,...
10006,99936,Luba Benser,274427,680,Carnage Blvd,685095054
10007,99941,Roxana Mckimley,975942,1613,Gate St,512136801
10008,99965,Cherie Zeimantz,287627,3661,The Water Ave,362877324
10009,99982,Allen Cruse,251350,3126,N Jean Dr,348734531


In [60]:
#identifico el primer testigo

query='''
SELECT *
FROM person
WHERE address_street_name = "Northwestern Dr"
ORDER BY address_number DESC;
'''
sql_query(query)

['14887', 'Morty Schapiro', '118009', '4919', 'Northwestern Dr', '111564949']
['17729', 'Lasonya Wildey', '439686', '3824', 'Northwestern Dr', '917817122']
['53890', 'Sophie Tiberio', '957671', '3755', 'Northwestern Dr', '442830147']
['73368', 'Torie Thalmann', '773862', '3697', 'Northwestern Dr', '341559436']
['96595', 'Coretta Cubie', '303645', '3631', 'Northwestern Dr', '378403829']
['19420', 'Cody Schiel', '890431', '3524', 'Northwestern Dr', '947110049']
['93509', 'Emmitt Aceuedo', '916706', '3491', 'Northwestern Dr', '979073160']
['87456', 'Leonora Wolfsberger', '215868', '3483', 'Northwestern Dr', '565203106']
['36378', 'Freddie Ellzey', '267882', '3449', 'Northwestern Dr', '474117596']
['53076', 'Boris Bijou', '664914', '3327', 'Northwestern Dr', '401191868']
['28360', 'Rashad Cascone', '924517', '3212', 'Northwestern Dr', '838907424']
['23044', 'Val Portlock', '924989', '3143', 'Northwestern Dr', '100593316']
['51114', 'Christena Saffell', '814284', '3055', 'Northwestern Dr', 

,id,name,license_id,address_number,address_street_name,ssn
0,14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949
1,17729,Lasonya Wildey,439686,3824,Northwestern Dr,917817122
2,53890,Sophie Tiberio,957671,3755,Northwestern Dr,442830147
3,73368,Torie Thalmann,773862,3697,Northwestern Dr,341559436
4,96595,Coretta Cubie,303645,3631,Northwestern Dr,378403829
5,19420,Cody Schiel,890431,3524,Northwestern Dr,947110049
6,93509,Emmitt Aceuedo,916706,3491,Northwestern Dr,979073160
7,87456,Leonora Wolfsberger,215868,3483,Northwestern Dr,565203106
8,36378,Freddie Ellzey,267882,3449,Northwestern Dr,474117596
9,53076,Boris Bijou,664914,3327,Northwestern Dr,401191868
